In [ ]:
# pip install deap

import numpy as np
import random
from deap import base, creator, tools, algorithms
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Define the XOR function
def xor(x, y):
    return int((x or y) and not (x and y))

# Function to evaluate the neural network's performance
def evaluate_nn(individual):
    model = create_nn(individual)
    predictions = model.predict(x_data)
    predictions = [int(round(p[0])) for p in predictions]
    return sum(p == t for p, t in zip(predictions, y_data)),

# Function to create a neural network based on the individual's genome
def create_nn(individual):
    model = Sequential()
    model.add(Dense(units=individual[0], activation='relu', input_shape=(2,)))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy')
    return model

# Genetic algorithm setup
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('attr_neurons', random.randint, 2, 10)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.attr_neurons, n=1)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('evaluate', evaluate_nn)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutUniformInt, low=2, up=10, indpb=0.2)
toolbox.register('select', tools.selTournament, tournsize=3)

# Generate some example data for XOR function
np.random.seed(42)
num_samples = 100
x_data = np.random.randint(0, 2, size=(num_samples, 2))
y_data = [xor(x[0], x[1]) for x in x_data]

# Main genetic algorithm loop
if __name__ == '__main__':
    population_size = 50
    generations = 20

    pop = toolbox.population(n=population_size)
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for gen in range(1, generations + 1):
        offspring = algorithms.varAnd(pop, toolbox, cxpb=0.5, mutpb=0.2)
        fits = toolbox.map(toolbox.evaluate, offspring)
        for ind, fit in zip(offspring, fits):
            ind.fitness.values = fit
        pop = toolbox.select(offspring, k=population_size)

        # Get the best individual in the current generation
        best_individual = tools.selBest(pop, k=1)[0]
        best_fitness = best_individual.fitness.values[0]
        print(f'Generation {gen}, Best Fitness: {best_fitness}')

    best_individual = tools.selBest(pop, k=1)[0]
    print('Best Individual:', best_individual)
